In [1]:
# import the necessary packages
import numpy as np
import argparse
import cv2
from matplotlib import pyplot as plt

# import packages to process video and images
import math
import time  

# import packages to write a json file
import os
import json

## Detecting color in videos using OpenCV

In [2]:
# getting objects from one frame based on color present
def get_objects_from_frame(original_img, target_objects):
    img = original_img.copy()[:,:,::-1]
    img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV) # convert the image from rgb to hsv. 
    detected_objects = []
    for target_obj in target_objects:
        (lower, upper) = target_obj['boundaries_hsv']
        # create NumPy arrays from the boundaries
        lower = np.array(lower, dtype = "uint8")
        upper = np.array(upper, dtype = "uint8")
        # find the colors within the specified boundaries and apply the mask
        mask = cv2.inRange(img_hsv, lower, upper)
        # color detected image
        output = cv2.bitwise_and(img, img, mask = mask)

        # count mask
        number_of_white_pix = np.sum(mask == 255) # find color. alternative: cv2.countNonZero(mask)
        number_of_black_pix = np.sum(mask == 0) # didn't find color
        color_pixel_percentage = number_of_white_pix*100 / (number_of_white_pix + number_of_black_pix)
        # print('Number of white pixels:', number_of_white_pix) # cv2.countNonZero(mask)
        # print('Number of black pixels:', number_of_black_pix)

        # identify detected object based on target color pixel percentage
        if(color_pixel_percentage > target_obj['pixel_percentage']):
            detected_objects.append(target_obj['name'])

        # plot results
#         plt.figure()
#         plt.subplot(121)
#         plt.imshow(img)
#         plt.subplot(122)
#         plt.imshow(mask, cmap='gray')
#         plt.imshow(output)

        # print(target_obj['name'], ' - Color pixel percentage:', np.round(color_pixel_percentage,3))
        
    return detected_objects

In [5]:


def get_timeStamp():
    unix_timestamp = time.time()  # Unix Timestamp: Applying time() method             
    unix_timestamp_ptg = str(int(unix_timestamp*1000))+"-0"  # convert to PTG timestamp format
    return unix_timestamp_ptg

"""
Getting a list of dictionaries from a video with the following info:
[
    {"frame_type":null,"values":[],"timestamp":"1678912603233-0"},
    {"frame_type":123,"values":[
        {"xyxyn":[0,0,0,0],"confidence":1,"class_id":1,"label":"CDU Interior"},
        {"xyxyn":[0,0,0,0],"confidence":1,"class_id":1,"label":"FDVCP"}
        ],
    "timestamp":"1678912604351-0"
    },
    ...
]
"""
# ([0,50,50], [10,255,255])
def get_json_values(videoFile):

    # boundaries_hsv: define color boundaries (lower and upper)
    # pixel_percentage: check if object is present based on color pixel percentage
    target_objects = [ 
        {"name": "CDU Interior", "color": 'red_hsv', "boundaries_hsv": ([0,140,115], [6,255,255]), "pixel_percentage": 0.2 },
        {"name": "Inboard MFD", 'color': 'blue_hsv', "boundaries_hsv": ([100,100,100], [130,255,255]), "pixel_percentage": 0.2 },
        {"name": "Outboard MFD", "color": 'green_hsv', "boundaries_hsv": ([25, 152, 172], [90, 255, 255]), "pixel_percentage": 0.2 },
        {"name": "FDVCP", "color": 'magenta_hsv', "boundaries_hsv": ([135,193,216], [156, 255, 255]), "pixel_percentage": 0.2 }
    ]
    #   cdu: [255, 0, 0] -> Red
    #   mfd inboard: [0, 0, 255] -> Blue
    #   mfd outboard: [0, 204, 0] -> Greem
    #   fdvcp: [252, 0, 204] -> Purple
 

    # Opens the Video file
    cap= cv2.VideoCapture(videoFile)
    json_values = []

    i=1
    frameRate = cap.get(5) #frame rate  
    while(cap.isOpened()):
        ret, frame = cap.read()
        frameId = cap.get(1) #current frame number
        if ret == False:
            break
        # if i%10 == 0: # get 10th frame
        if (frameId % math.floor(frameRate) == 0): # get one frame per second
            # cv2.imwrite('./frames/kang'+str(i)+'.jpg',frame) # save frame as image
            img = frame
            # detected_objects = get_objects_from_frame(img[:,:,::-1], boundaries_hsv, pixel_percentage, objects, i)
            detected_objects = get_objects_from_frame(img, target_objects) #boundaries_hsv, pixel_percentage, objects)
            print(detected_objects)

            # PTG format
            # add frame info
            values = []
            objects = map(lambda x: x["name"], target_objects)
            for obj in objects:
                if(obj in detected_objects):
                    values.append({"xyxyn":[0,0,0,0], "confidence":1,"class_id":1,"label":obj})

            json_values.append({"frame_type":123, "values": values, "timestamp":get_timeStamp()})

        i+=1  

    cap.release()
    cv2.destroyAllWindows()
    
    return json_values

In [6]:
filename = 'detic_image_improve_red_detection_v2.json' # 'detic-image-fixed-labels.json'
isFile = os.path.isfile(filename)

json_data = get_json_values('hl2_rgb.mp4')

if (not isFile):
    with open(filename, 'w') as fp:
        json.dump(json_data, fp, indent=4)
else:
    print(f'An error occurred writing to {filename}.') 

['CDU Interior', 'Outboard MFD', 'FDVCP']
['Inboard MFD']
['CDU Interior', 'Inboard MFD', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Inboard MFD', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Outboard MFD']
['CDU Interior', 'Inboard MFD', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Outboard MFD', 'FDVCP']
['Outboard MFD']
['Inboard MFD']
['Inboard MFD']
['Inboard MFD']
['Inboard MFD']
['Inboard MFD']
['CDU Interior', 'Inboard MFD', 'Outboard MFD']
[]
['FDVCP']
[]
[]
[]
[]
[]
[]
['Outboard MFD']

['CDU Interior', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Inboard MFD', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Inboard MFD', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Inboard MFD', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Inboard MFD', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Inboard MFD', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Inboard MFD', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Inboard MFD', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Inboard MFD', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Inboard MFD', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Inboard MFD', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Inboard MFD', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Inboard MFD', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Inboard MFD', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Inboard MFD', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Inboard MFD', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Inboard MFD', 'Outboard MFD', 'FDVCP']
['CDU Interior', 'Inboard MFD', 'Outboard MFD'